# Hall of Mirrors 3 (2025 March Jane Street puzzle)

https://www.janestreet.com/puzzles/hall-of-mirrors-3-index/

## 0. The solution

We place the mirrors according to the numbers given along the borders, using recursion with backtracking to explore all valid configurations of mirrors consistent with the given border values.

For each given number $v$ on the edge and for each of its divisors $d$, we launch a ray of length $d$ and place a mirror at its endpoint. When we place a mirror (either `/` or `\\`), the ray is reflected, and we launch a new ray using the divisors of $v/d$.

When a ray reaches another border, that path is valid only if it arrives with value $1$, and, additionally, if the cell it reaches already contains a given number, that number must equal the initial value $v$. Once this condition is satisfied, we proceed to the next given number on the border.

During the ray-launching process, the following constraints must always be checked:
* a mirror cannot be placed adjacent to another existing mirror,  
* rays cannot cross existing mirrors (the "path must be clear"),  
* mirrors cannot be placed on cells already used by a previous ray (the set `used_cells` keeps track of all cells already traversed by previous rays to prevent overlaps during the search), and  
* if a mirror is to be placed on a cell where one already exists, we simply reflect the ray according to that mirror's orientation (we do not attempt both orientations, since the existing mirror determines it).

In [1]:
import sympy as sp

# ANSI color codes
BLUE = '\033[94m'
GREEN = '\033[92m'
RED = '\033[91m'
RESET = '\033[0m'

In [2]:
def print_hall_of_mirrors(grid, N=12):
    cell_w = 5  # cell width
    inner_N = N - 2  # playable (N-2)x(N-2) area

    # Extract edge clues
    top = grid[0]
    bottom = grid[N - 1]
    left = [grid[r][0] for r in range(1, N - 1)]
    right = [grid[r][N - 1] for r in range(1, N - 1)]

    # Helper: one horizontal border
    def horizontal_border():
        return " " * (cell_w + 1) + "+" + "+".join("-" * cell_w for _ in range(inner_N)) + "+"

    # Top numbers
    top_line = " " * (cell_w - 4)
    for c, value in enumerate(top):
        if value in {'/', '\\'}:
            top_line += f"{BLUE}{str(value):^{cell_w}}{RESET} "
        elif (0, c) not in given:
            top_line += f"{RED}{str(value):^{cell_w}}{RESET} "
        else:
            top_line += f"{str(value) if value else '':^{cell_w}} "
    print(top_line)
    print(horizontal_border())

    # Rows of inner grid
    for i, r in enumerate(range(1, N - 1)):
        # Left clue
        left_value = left[i]
        if left_value in {'/', '\\'}:
            left_clue = f"{BLUE}{str(left_value):>{cell_w}}{RESET}"
        elif (r, 0) not in given:
            left_clue = f"{RED}{str(left_value):>{cell_w}}{RESET}"
        else:
            left_clue = f"{str(left_value) if left_value else '':>{cell_w}}"

        # Right clue
        right_value = right[i]
        if right_value in {'/', '\\'}:
            right_clue = f"{BLUE}{str(right_value):<{cell_w}}{RESET}"
        elif (r, N-1) not in given:
            right_clue = f"{RED}{str(right_value):<{cell_w}}{RESET}"
        else:
            right_clue = f"{str(right_value) if right_value else '':<{cell_w}}"

        # Playable cells (inner grid)
        row_cells = ""
        for c in range(1, N - 1):
            cell_value = grid[r][c]
            if cell_value in {'/', '\\'}:
                row_cells += f"{BLUE}{str(cell_value):^{cell_w}}{RESET}"
            else:
                row_cells += f"{str(cell_value) if isinstance(cell_value, str) else '':^{cell_w}}"
            if c < N - 2:
                row_cells += "|"

        print(f"{left_clue} |{row_cells}| {right_clue}")
        print(horizontal_border())

    # Bottom numbers
    bottom_line = " " * (cell_w - 4)
    for c, value in enumerate(bottom):
        if value in {'/', '\\'}:
            bottom_line += f"{BLUE}{str(value):^{cell_w}}{RESET} "
        elif (N-1, c) not in given:
            bottom_line += f"{RED}{str(value):^{cell_w}}{RESET} "
        else:
            bottom_line += f"{str(value) if value else '':^{cell_w}} "
    print(bottom_line)

In [3]:
# ===============================
# Helper functions
# ===============================

def in_bounds(r, c):
    """Check if (r,c) is inside grid bounds."""
    return 0 <= r < N and 0 <= c < N


def reflect(direction, mirror):
    """Return new direction after reflection."""
    if mirror == '/':
        return {'up': 'right', 'right': 'up', 'down': 'left', 'left': 'down'}[direction]
    else:  # mirror == '\\'
        return {'up': 'left', 'left': 'up', 'down': 'right', 'right': 'down'}[direction]

def adjacent_has_mirror(r, c):
    """Return True if any orthogonal neighbor of (r,c) has a mirror."""
    for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]:
        rr, cc = r + dr, c + dc
        if in_bounds(rr, cc) and grid[rr][cc] in mirrors:
            return True
    return False

In [4]:
# ===============================
# Core recursion
# ===============================

def launch_rays(index, r, c, v, v_init, direction, first_call=False):
    """
    Propagate a ray from (r, c) with current value v and original value v_init.
    - Marks traversed cells in `used_cells`
    - Avoids placing mirrors on used cells
    - Backtracks properly
    """
    if not in_bounds(r, c):
        return False

    # --- Border check ---
    if (r == 0 or r == N - 1 or c == 0 or c == N - 1):
        if first_call:
            # starting from the border — no v==1 check yet
            pass
        else:
            # hitting a border again — must finish with v=1
            if v != 1:
                return False
            if (r, c) in given_index and given[(r, c)] != v_init:
                return False
            return solve(index + 1)

    # --- Try divisors of v ---
    for d in sp.divisors(v):
        # Step direction
        dr, dc = 0, 0
        if direction == 'up': dr, dc = -1, 0
        elif direction == 'down': dr, dc = 1, 0
        elif direction == 'left': dr, dc = 0, -1
        elif direction == 'right': dr, dc = 0, 1

        r_next, c_next = r + dr * d, c + dc * d
        if not in_bounds(r_next, c_next) or (r_next, c_next) in used_cells:
            continue  # cannot place a mirror on a used cell

        # --- Check if path is clear ---
        path_cells = [(r + dr * j, c + dc * j) for j in range(1, d)]
        if any(grid[rr][cc] in mirrors for rr, cc in path_cells):
            continue  # cannot cross mirrors

        # --- Mark path as used (only new ones) ---
        newly_added = [cell for cell in path_cells if cell not in used_cells]
        for cell in newly_added:
            used_cells.add(cell)

        # --- Case 1: next cell hits border ---
        if r_next == 0 or r_next == N-1 or c_next == 0 or c_next == N-1:
            # continue in SAME direction (no mirror)
            success = launch_rays(index, r_next, c_next, v // d, v_init, direction)
            if success:
                return True
        elif grid[r_next][c_next] in mirrors: # --- Case 2: the new one is already a mirror ---
            m = grid[r_next][c_next]
            new_dir = reflect(direction, m)
            success = launch_rays(index, r_next, c_next, v // d, v_init, new_dir)
            if success:
                return True
        else:
            # --- Case 3: try both mirror placements ---
            if not adjacent_has_mirror(r_next, c_next): # forbid adjacent mirrors
                for mirror in ('/', '\\'):
                    new_dir = reflect(direction, mirror)
                    grid[r_next][c_next] = mirror

                    success = launch_rays(index, r_next, c_next, v // d, v_init, new_dir)
                    if success:
                        return True

                    # Backtrack
                    grid[r_next][c_next] = ""

        # --- Backtrack: remove only newly added path cells ---
        for cell in newly_added:
            used_cells.remove(cell)

    return False


def solve(index=0):
    """Recursive solver over all border clues."""
    if index == len(given_index):
        #solution found
        return True

    r, c = given_index[index]
    v_init = given[(r, c)]

    if r == 0: direction = 'down'
    elif r == N - 1: direction = 'up'
    elif c == 0: direction = 'right'
    elif c == N - 1: direction = 'left'
    else:
        return False  # invalid starting point

    #print(f"\nLaunching ray {index}: start=({r},{c}), value={v_init}, dir={direction}")
    return launch_rays(index, r, c, v_init, v_init, direction, first_call=True)


In [5]:
def compute_ray(r,c,direction,starting_ray = False):
    if not starting_ray and (r==0 or r==N-1 or c==0 or c==N-1):
        return 1

    # Step direction
    dr, dc = 0, 0
    if direction == 'up': dr, dc = -1, 0
    elif direction == 'down': dr, dc = 1, 0
    elif direction == 'left': dr, dc = 0, -1
    elif direction == 'right': dr, dc = 0, 1

    i = 1
    r_next, c_next = r + dr * i, c + dc * i
    while not (r_next==0 or r_next==N-1 or c_next==0 or c_next==N-1) and grid[r_next][c_next] not in mirrors:
        i += 1
        r_next, c_next = r + dr * i, c + dc * i

    return i * compute_ray(r_next, c_next, reflect(direction, grid[r_next][c_next]))


In [7]:
# ===============================
# Example setup and run
# ===============================

mirrors = {'/', '\\'}
used_cells = set()  # track ray paths


N = 12
grid = [['' for _ in range(N)] for _ in range(N)]
given = {
    (0,3):112, (0,5):48, (0,6):3087, (0,7):9, (0,10):1,
    (2,11):4,
    (3,11):27,
    (4,0):27,
    (7,11):16,
    (8,0):12,
    (9,0):225,
    (11,1):2025, (11,4):12, (11,5):64, (11,6):5, (11,8):405
}

'''
N = 7
grid = [['' for _ in range(N)] for _ in range(N)]
given = {(0,3):9, (2,6):75, (4,0):16, (6,3):36}
'''

given_index = list(given.keys())

for (r, c), v in given.items():
    grid[r][c] = v

print("Looking for a solution for the following configuration:\n")
print_hall_of_mirrors(grid, N)
print("\n... launching some rays ...\n")
found = solve()
if found:
    print("✅ SOLUTION FOUND!")
    print_hall_of_mirrors(grid, N)
else:
    print("No solution found.")


# ----------------------------------------------------------------------
# Let's now fill in the missing values
# ----------------------------------------------------------------------


top = 0
bottom = 0
left = 0
right = 0
for aux in range(1,N-1):
    if (0,aux) not in given_index:
        v = compute_ray(0,aux,'down',starting_ray=True)
        grid[0][aux] = v
        top += v
    if (N-1,aux) not in given_index:
        v = compute_ray(N-1,aux,'up',starting_ray=True)
        grid[N-1][aux] = v
        bottom += v
    if (aux,0) not in given_index:
        v = compute_ray(aux,0,'right',starting_ray=True)
        grid[aux][0] = v
        left += v
    if (aux,N-1) not in given_index:
        v = compute_ray(aux,N-1,'left',starting_ray=True)
        grid[aux][N-1] = v
        right += v

print("\n... computing the missing values ...\n")
print_hall_of_mirrors(grid,N)

print(f"\nThe solution is {GREEN}{top*right*bottom*left} = {top} (top) x {right} (right) x {bottom} (bottom) x {left} (left){RESET}")

Looking for a solution for the following configuration:

                    112         48   3087    9                 1         
      +-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
      |     |     |     |     |     |     |     |     |     |     |      
      +-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
      |     |     |     |     |     |     |     |     |     |     | 4    
      +-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
      |     |     |     |     |     |     |     |     |     |     | 27   
      +-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
   27 |     |     |     |     |     |     |     |     |     |     |      
      +-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
      |     |     |     |     |     |     |     |     |     |     |      
      +-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
      |     |     |     |     |     |     |     |     |     |     |      
      +-----+----